In [1]:
from chunking_evaluation import GeneralEvaluation, SyntheticEvaluation, BaseChunker
from chromadb.utils import embedding_functions

import pandas as pd
from IPython.display import display, clear_output
import os

In [2]:
corpora_paths = [r"nvidia_10k_utf8.txt"]
csv_path = r"generated_queries_and_excerpts.csv"

synthetic_pipeline = SyntheticEvaluation(
    corpora_paths,
    csv_path,
    openai_api_key=os.environ["OPENAI_API_KEY"]
)


In [3]:
synthetic_df = pd.read_csv(csv_path)
synthetic_df

,question,references,corpus_id
0,What is the net income of NVIDIA Corporation f...,"[{""content"": ""tax expense (benefit) 4,058 (187...",./domain_specific/nvidia_10k.txt
1,What are the consequences of not adhering to d...,"[{""content"": ""ct to penalties of up to \u20ac2...",./domain_specific/nvidia_10k.txt
2,What are the steps involved in revenue recogni...,"[{""content"": ""share.\nRevenue Recognition\nWe ...",./domain_specific/nvidia_10k.txt
3,How are operating lease assets and liabilities...,"[{""content"": ""lease payments over the lease te...",./domain_specific/nvidia_10k.txt
4,What are the conditions under which the Compan...,"[{""content"": ""provide it to Nasdaq in accordan...",./domain_specific/nvidia_10k.txt
...,...,...,...
61,What are the conditions necessary for recogniz...,"[{""content"": ""benefits during the period.\nWe ...",./domain_specific/nvidia_10k.txt
62,What subsidiaries does NVIDIA Corporation own?,"[{""content"": ""a significant subsidiary.\nSubsi...",./domain_specific/nvidia_10k.txt
63,What are the significant changes in Other Inco...,"[{""content"": ""prepayment provided at\nsigning....",./domain_specific/nvidia_10k.txt
64,What are the recent changes in the share repur...,"[{""content"": ""Shareholders\u2019 Equity\nCapit...",./domain_specific/nvidia_10k.txt


In [4]:
synthetic_pipeline.generate_queries_and_excerpts(approximate_excerpts=True, 
                                         num_rounds=1, 
                                         queries_per_corpus=5)

Trying Query 0
Trying Query 1
Trying Query 2
Trying Query 3
Trying Query 4


Filter Poor Excerpts

In [7]:
synthetic_pipeline.filter_poor_excerpts(threshold=0.36)

Corpus: ./domain_specific/nvidia_10k.txt - Removed 0 .
Corpus: nvidia_10k_utf8.txt - Removed 1 .


Remove Duplicates

In [8]:
synthetic_pipeline.filter_duplicates(threshold=0.7)

Corpus: ./domain_specific/nvidia_10k.txt - Removed 0 .
Corpus: nvidia_10k_utf8.txt - Removed 0 .


In [11]:
synthetic_df = pd.read_csv(csv_path)
synthetic_df

,question,references,corpus_id
0,What is the net income of NVIDIA Corporation f...,"[{""content"": ""tax expense (benefit) 4,058 (187...",./domain_specific/nvidia_10k.txt
1,What are the consequences of not adhering to d...,"[{""content"": ""ct to penalties of up to \u20ac2...",./domain_specific/nvidia_10k.txt
2,What are the steps involved in revenue recogni...,"[{""content"": ""share.\nRevenue Recognition\nWe ...",./domain_specific/nvidia_10k.txt
3,How are operating lease assets and liabilities...,"[{""content"": ""lease payments over the lease te...",./domain_specific/nvidia_10k.txt
4,What are the conditions under which the Compan...,"[{""content"": ""provide it to Nasdaq in accordan...",./domain_specific/nvidia_10k.txt
...,...,...,...
65,What does NVIDIA's full-stack computing infras...,"[{""content"": ""offerings that are reshaping ind...",./domain_specific/nvidia_10k.txt
66,What financial risks was the company exposed t...,"[{""content"": ""us, or our ability to fulfill ou...",nvidia_10k_utf8.txt
67,What are the details of NVIDIA Corporation's i...,"[{""content"": ""(2,165) (420)\nState (206) \u00e...",nvidia_10k_utf8.txt
68,What are the applications of NVIDIA RTX techno...,"[{""content"": ""broadcast-\ntelevision graphics....",nvidia_10k_utf8.txt


In [15]:
from typing import List
class SentenceChunker(BaseChunker):
    def __init__(self, sentences_per_chunk: int = 3):
        # Initialize the chunker with the number of sentences per chunk
        self.sentences_per_chunk = sentences_per_chunk

    def split_text(self, text: str) -> List[str]:
        # Handle the case where the input text is empty
        if not text:
            return []

        # Split the input text into sentences using regular expression
        # Regex looks for white space following . ! or ? and makes a split
        sentences = re.split(r'(?<=[.!?])\s+', text)
        chunks = []

        # Group sentences into chunks based on the specified number
        for i in range(0, len(sentences), self.sentences_per_chunk):
            # Combine sentences into a single chunk
            chunk = ' '.join(sentences[i:i + self.sentences_per_chunk])
            chunks.append(chunk)
        
        # Return the list of chunks
        return chunks

In [16]:
sentence_chunker = SentenceChunker(sentences_per_chunk = 10)

# Define OpenAI Embedding Model
default_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.environ["OPENAI_API_KEY"],
    model_name="text-embedding-3-large"
)

# Run the Eval With Chunker and Embedding Model on our Synthetic Dataset
synth_results = synthetic_pipeline.run(sentence_chunker, default_ef)

# Display our Results
print_metrics(synth_results)

NotFoundError: Collection [auto_chunk] does not exist

In [17]:
# Create the collection once so delete_collection("auto_chunk") won't crash
_ = synthetic_pipeline.chroma_client.get_or_create_collection(
    name="auto_chunk",
    embedding_function=default_ef,
)

# Now run evaluation
synth_results = synthetic_pipeline.run(sentence_chunker, default_ef)
print_metrics(synth_results)


FileNotFoundError: [Errno 2] No such file or directory: './domain_specific/nvidia_10k.txt'